In [1]:
import os
import pandas as pd
from pathlib import Path
from typing import Dict, List, Any

In [2]:
def extract_sheets_to_csv(excel_file_path: str, output_dir: str = "output") -> Dict[str, str]:
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    excel_file = pd.ExcelFile(excel_file_path, engine='openpyxl')
    csv_files = {}

    for sheet_name in excel_file.sheet_names:
        df = pd.read_excel(excel_file_path, sheet_name=sheet_name, engine='openpyxl')

        safe_sheet_name = "".join(c for c in sheet_name if c.isalnum() or c in ['_', '-', '.']).rstrip('')
        csv_filename = f"{safe_sheet_name}.csv"
        csv_path = os.path.join(output_dir, csv_filename)
        df.to_csv(csv_path, index=False)
        csv_files[sheet_name] = csv_path

    excel_file.close()
    return csv_files

In [4]:
def analyze_dataframe(df: pd.DataFrame, sheet_name: str) -> Dict[str, Any]:
    analysis = {
        'sheet_name': sheet_name,
        'shape': df.shape,
        'columns': df.columns.tolist(),
        'data_types': df.dtypes.to_dict(),
        'null_counts': df.isnull().sum().to_dict(),
        'memory_usage': df.memory_usage(deep=True).sum(),
        'numeric_columns': df.select_dtypes(include=['number']).columns.tolist(),
        'categorical_columns': df.select_dtypes(include=['object', 'category']).columns.tolist(),
        'datetime_columns': df.select_dtypes(include=['datetime']).columns.tolist()
    }
    
    if analysis['numeric_columns']:
        analysis['numeric_summary'] = df[analysis['numeric_columns']].describe().to_dict()
    
    if analysis['categorical_columns']:
        analysis['categorical_summary'] = {}
        for col in analysis['categorical_columns']:
            analysis['categorical_summary'][col] = {
                'unique_count': df[col].nunique(),
                'top_values': df[col].value_counts().head(5).to_dict()
            }
    
    analysis['duplicate_rows'] = df.duplicated().sum()
    
    return analysis

In [5]:
def analyze_csv_files(csv_files: Dict[str, str]) -> Dict[str, Dict[str, Any]]:
    all_analyses = {}
    
    for sheet_name, csv_path in csv_files.items():
        df = pd.read_csv(csv_path)
        analysis = analyze_dataframe(df, sheet_name)
        all_analyses[sheet_name] = analysis
        
    return all_analyses


In [6]:
def print_analysis_summary(analyses: Dict[str, Dict[str, Any]]) -> None:
    print(f"Analysis Summary for {len(analyses)} sheets:")
    print("=" * 50)
    
    for sheet_name, analysis in analyses.items():
        print(f"\nSheet: {sheet_name}")
        print(f"  Shape: {analysis['shape'][0]} rows × {analysis['shape'][1]} columns")
        print(f"  Memory Usage: {analysis['memory_usage'] / 1024:.2f} KB")
        print(f"  Duplicate Rows: {analysis['duplicate_rows']}")
        
        if analysis['numeric_columns']:
            print(f"  Numeric Columns: {len(analysis['numeric_columns'])}")
        
        if analysis['categorical_columns']:
            print(f"  Categorical Columns: {len(analysis['categorical_columns'])}")
        
        null_cols = [col for col, count in analysis['null_counts'].items() if count > 0]
        if null_cols:
            print(f"  Columns with Null Values: {len(null_cols)}")


In [7]:
def generate_detailed_report(analyses: Dict[str, Dict[str, Any]], output_file: str = "analysis_report.txt") -> None:
    with open(output_file, 'w') as f:
        f.write("Detailed Excel Workbook Analysis Report\n")
        f.write("=" * 50 + "\n\n")
        
        for sheet_name, analysis in analyses.items():
            f.write(f"SHEET: {sheet_name}\n")
            f.write("-" * 30 + "\n")
            f.write(f"Dimensions: {analysis['shape'][0]} rows × {analysis['shape'][1]} columns\n")
            f.write(f"Memory Usage: {analysis['memory_usage'] / 1024:.2f} KB\n")
            f.write(f"Duplicate Rows: {analysis['duplicate_rows']}\n\n")
            
            f.write("COLUMNS AND DATA TYPES:\n")
            for col, dtype in analysis['data_types'].items():
                null_count = analysis['null_counts'][col]
                f.write(f"  {col}: {dtype} (Nulls: {null_count})\n")
            
            if analysis.get('numeric_summary'):
                f.write("\nNUMERIC SUMMARY:\n")
                for col, stats in analysis['numeric_summary'].items():
                    f.write(f"  {col}:\n")
                    f.write(f"    Mean: {stats.get('mean', 'N/A'):.2f}\n")
                    f.write(f"    Std: {stats.get('std', 'N/A'):.2f}\n")
                    f.write(f"    Min: {stats.get('min', 'N/A')}\n")
                    f.write(f"    Max: {stats.get('max', 'N/A')}\n")
            
            if analysis.get('categorical_summary'):
                f.write("\nCATEGORICAL SUMMARY:\n")
                for col, summary in analysis['categorical_summary'].items():
                    f.write(f"  {col}: {summary['unique_count']} unique values\n")
                    f.write(f"    Top values: {summary['top_values']}\n")
            
            f.write("\n" + "="*50 + "\n\n")


def process_excel_workbook(excel_file_path: str, output_directory: str = "extracted_sheets") -> Dict[str, Dict[str, Any]]:
    print(f"Processing Excel file: {excel_file_path}")
    
    csv_files = extract_sheets_to_csv(excel_file_path, output_directory)
    print(f"Extracted {len(csv_files)} sheets to CSV files in '{output_directory}' directory")
    
    analyses = analyze_csv_files(csv_files)
    print_analysis_summary(analyses)
    
    generate_detailed_report(analyses)
    print("Detailed report saved as 'analysis_report.txt'")
    
    return analyses

In [8]:
excel_file_path = Path('__file__').parent / 'data' / 'alm.xlsx'
if not os.path.exists(excel_file_path):
    raise FileNotFoundError(f"The file {excel_file_path} does not exist")

try:
    analyses = process_excel_workbook(excel_file_path)
except Exception as e:
    print(f"An error occurred: {e}")
    print("Please check the file path and permissions.")



Processing Excel file: data/alm.xlsx
Extracted 2 sheets to CSV files in 'extracted_sheets' directory
Analysis Summary for 2 sheets:

Sheet: ALM
  Shape: 190 rows × 19 columns
  Memory Usage: 163.63 KB
  Duplicate Rows: 1
  Numeric Columns: 3
  Categorical Columns: 16
  Columns with Null Values: 19

Sheet: IRS
  Shape: 249 rows × 17 columns
  Memory Usage: 186.63 KB
  Duplicate Rows: 8
  Numeric Columns: 3
  Categorical Columns: 14
  Columns with Null Values: 17
Detailed report saved as 'analysis_report.txt'


In [9]:
next_excel_file_path = Path('__file__').parent / 'data' / 'operations.xlsx'
try:
    analyses = process_excel_workbook(next_excel_file_path)
except Exception as e:
    print(f"An error occurred: {e}")
    print("Please check the file path and permissions.")





Processing Excel file: data/operations.xlsx
Extracted 3 sheets to CSV files in 'extracted_sheets' directory
Analysis Summary for 3 sheets:

Sheet: Sheet1
  Shape: 96 rows × 12 columns
  Memory Usage: 40.71 KB
  Duplicate Rows: 15
  Numeric Columns: 3
  Categorical Columns: 9
  Columns with Null Values: 12

Sheet: Sheet2
  Shape: 25 rows × 14 columns
  Memory Usage: 10.44 KB
  Duplicate Rows: 2
  Numeric Columns: 6
  Categorical Columns: 8
  Columns with Null Values: 14

Sheet: Sheet3
  Shape: 6 rows × 4 columns
  Memory Usage: 0.64 KB
  Duplicate Rows: 0
  Numeric Columns: 3
  Categorical Columns: 1
Detailed report saved as 'analysis_report.txt'
